In [3]:

import numpy as np
from scipy.linalg import qr, lu, ldl


In [ ]:
# Week 3

# Week 3

In [4]:

def EqualityQPSolver(H, g, A, b, Method):
    # Construct the KKT matrix
    KKT_matrix = np.block([
        [H, -A.T],
        [-A, np.zeros((A.shape[0], A.shape[0]))]
    ])
    
    # Construct the right-hand side vector
    rhs = np.concatenate([-g, b])
    
    # Initialize the solution vector
    x = np.zeros(H.shape[1])
    
    # Choose the factorization method
    if Method == 'QR':
        Q, R = qr(KKT_matrix)
        F = R
    elif Method == 'LDL':
        L, D, perm = ldl(KKT_matrix)
        F = L @ D @ L.T
    else:
        P, L, U = lu(KKT_matrix)
        F = U
    
    # Solve the system
    solution = np.linalg.solve(F, rhs)
    
    # Extract x and lambda from the solution
    x = solution[:H.shape[1]]
    lamda = solution[H.shape[1]:]
    
    return x, lamda

In [6]:
# Define the problem data
H = np.array([
    [6, 1, 0.5],
    [1, 5, 1],
    [0.5, 1, 4]
])
g = np.array([-8, -3, -3])
A = np.array([
    [1, 0, 1],
    [0, 1, 1]
])
b = np.array([3, 0])

# Solve the problem using different methods
methods = ['QR', 'LDL', 'LU']
for method in methods:
    x, lamda = EqualityQPSolver(H, g, A, b, method)
    print(f"Method: {method}")
    print(f"Optimal x: {x}")
    print(f"Lagrange multipliers: {lamda}\n")

Method: QR
Optimal x: [ 0.03567243 -0.86156841  0.78731331]
Lagrange multipliers: [ 7.14536839 -0.        ]

Method: LDL
Optimal x: [-1.03571429  1.96428571 -1.96428571]
Lagrange multipliers: [-13.23214286   3.82142857]

Method: LU
Optimal x: [ 0.02221611  1.07034797 -1.04062263]
Lagrange multipliers: [-7.31666667 -0.        ]



In [5]:










import sympy as sp
import numpy as np

def construct_lagrangian(f, constraints_eq=[]):
    """
    Constructs the Lagrangian function for a given function f in R^n with equality constraints.
    """
    # Extract variables from the function
    variables = list(f.free_symbols)

    # Define Lagrange multipliers for equality constraints
    lambdas_eq = [sp.Symbol(f"λ{i+1}") for i in range(len(constraints_eq))]

    # Ensure f is a scalar expression (extract first element if f is a matrix)
    if isinstance(f, sp.Matrix) and f.shape == (1, 1):  # Check if f is a 1x1 matrix
        f = f[0]  # Extract scalar from matrix

    # Construct the Lagrangian function
    L = f + sum(lambdas_eq[i] * constraints_eq[i] for i in range(len(constraints_eq)))

    return L, variables, lambdas_eq

def newtons_method(F, J, variables, initial_guess, tol=1e-6, max_iter=100):
    """
    Uses Newton's method to find the roots of the system F = 0.
    """
    # Convert symbolic expressions to numerical functions
    F_func = sp.lambdify(variables, F, "numpy")
    J_func = sp.lambdify(variables, J, "numpy")

    # Initialize solution
    x_k = np.array(initial_guess, dtype=np.float64)

    for i in range(max_iter):
        F_k = np.array(F_func(*x_k), dtype=np.float64).flatten()
        J_k = np.array(J_func(*x_k), dtype=np.float64)

        # Compute Newton step
        try:
            delta_x = np.linalg.solve(J_k, -F_k)
        except np.linalg.LinAlgError:
            print("Jacobian is singular. Newton’s method failed.")
            return None

        # Update solution
        x_k = x_k + delta_x

        # Check convergence
        if np.linalg.norm(delta_x) < tol:
            return x_k

    print("Newton’s method did not converge within the max iteration limit.")
    return None

def find_optimal_points(f, constraints_eq, initial_guess=None):
    """
    Finds the optimal points using Newton’s method.
    """
    # Compute Lagrangian
    L, variables, lambdas_eq = construct_lagrangian(f, constraints_eq)

    # Full variable list (decision variables + Lagrange multipliers)
    all_vars = variables + lambdas_eq

    # Compute the gradient of the Lagrangian
    grad_L = [sp.diff(L, var) for var in all_vars]

    # Compute Jacobian matrix of the gradient system
    J = sp.Matrix(grad_L).jacobian(all_vars)

    # Set a default initial guess if none is provided
    if initial_guess is None:
        initial_guess = [1.0] * len(all_vars)

    # Use Newton’s method to solve the system
    solution = newtons_method(grad_L, J, all_vars, initial_guess)

    return solution, L, variables

def compute_hessian_at_optimum(L, variables, solution):
    """
    Computes the Hessian matrix of the Lagrangian at the optimal point found.

    Parameters:
    - L: Lagrangian function
    - variables: List of decision variables (excluding Lagrange multipliers)
    - solution: The optimal point found using Newton’s method

    Returns:
    - Hessian matrix evaluated at the optimal point
    - Eigenvalues at the optimal point
    - Classification of the stationary point
    """
    # Compute Hessian matrix
    H = sp.Matrix([[sp.diff(L, var1, var2) for var2 in variables] for var1 in variables])

    # Convert Hessian to a numerical function
    H_func = sp.lambdify(variables, H, "numpy")

    # Evaluate Hessian at the optimal solution
    if solution is None:
        print("No solution found, skipping Hessian computation.")
        return None, None, "No solution found"

    # Extract only the variable values (ignore Lagrange multipliers)
    var_solution = solution[:len(variables)]

    # Compute numerical Hessian
    H_eval = np.array(H_func(*var_solution), dtype=np.float64)

    # Compute eigenvalues
    eigenvalues = np.linalg.eigvals(H_eval)

    # Analyze eigenvalues
    positive = np.all(eigenvalues > 0)
    negative = np.all(eigenvalues < 0)

    if positive:
        classification = "Local minimum (all eigenvalues are positive)"
    elif negative:
        classification = "Local maximum (all eigenvalues are negative)"
    else:
        classification = "Saddle point (eigenvalues have mixed signs)"

    return H_eval, eigenvalues, classification

Method: QR
Optimal x: [ 0.03567243 -0.86156841  0.78731331]
Lagrange multipliers: [ 7.14536839 -0.        ]

Method: LDL
Optimal x: [-1.03571429  1.96428571 -1.96428571]
Lagrange multipliers: [-13.23214286   3.82142857]

Method: LU
Optimal x: [ 0.02221611  1.07034797 -1.04062263]
Lagrange multipliers: [-7.31666667 -0.        ]



ModuleNotFoundError: No module named 'sympy'